<a href="https://colab.research.google.com/github/Karla-Flores/Big-Data--Challenge/blob/main/Level2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os
# Find the latest version of spark 3.0  from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.0.3'
spark_version = 'spark-3.1.2'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Get:1 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:2 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Ign:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:4 http://archive.ubuntu.com/ubuntu bionic InRelease
Ign:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:6 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:7 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:8 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Get:9 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Get:10 http://security.ubuntu.com/ubuntu bionic-security/restricted amd64 Packages [581 kB]
Hit:11 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Get:12 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 

In [2]:
# For connection to Postgres
!wget https://jdbc.postgresql.org/download/postgresql-42.2.9.jar

--2021-09-24 17:11:48--  https://jdbc.postgresql.org/download/postgresql-42.2.9.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 914037 (893K) [application/java-archive]
Saving to: ‘postgresql-42.2.9.jar’

postgresql-42.2.9.j 100%[===================>] 892.61K  --.-KB/s    in 0.1s    

2021-09-24 17:11:48 (6.03 MB/s) - ‘postgresql-42.2.9.jar’ saved [914037/914037]



In [3]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("CloudETL").config("spark.driver.extraClassPath","/content/postgresql-42.2.9.jar").getOrCreate()

x

In [4]:
# Read in data
# Instructions mention that Tab ('\t')
from pyspark import SparkFiles
url='https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Luggage_v1_00.tsv.gz'
spark.sparkContext.addFile(url)
review_df = spark.read.csv(SparkFiles.get('amazon_reviews_us_Luggage_v1_00.tsv.gz'), sep='\t', header=True, inferSchema=True)

# Show DataFrame
review_df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   40884699| R9CO86UUJCAW5|B00VGTN02Y|     786681372|Teenage Mutant Ni...|         Luggage|          3|            0|          0|   N|                Y|my review of this...|my review of this...| 2015-08-31|
|         US|   23208852|R3PR8X6QGVJ8B1|B005KIWL0E|     618251799|Kenneth Cole Reac...|         Luggage|          5|    

In [33]:
review_df.count()

348657

In [57]:
# Drop null values
dropna_df = review_df.dropna()
dropna_df.count()

348613

In [47]:
# Load in a sql function to use columns
from pyspark.sql.functions import col

# Filter for only columns with active users
review_id_table_df =dropna_df.select(['review_id','product_id', 'star_rating','total_votes','vine','review_headline', 'review_body'])
review_id_table_df.show()

+--------------+----------+-----------+-----------+----+--------------------+--------------------+
|     review_id|product_id|star_rating|total_votes|vine|     review_headline|         review_body|
+--------------+----------+-----------+-----------+----+--------------------+--------------------+
| R9CO86UUJCAW5|B00VGTN02Y|          3|          0|   N|my review of this...|my review of this...|
|R3PR8X6QGVJ8B1|B005KIWL0E|          5|          0|   N|          Five Stars|       Perfect size.|
|R39BO2819ABUPF|B007UNSHJ6|          4|          0|   N|            so good.|    So far, so good.|
|R3ADL7V6EGGEEP|B00WHFNXB4|          4|          0|   N|          Four Stars|seems nicely made...|
|R1OXYPBPLVRMI5|B000M5NBYU|          5|          0|   N|           Dance Bag|Great bag for all...|
|R1WYM8Z5ATQ98O|B00ACBJ89G|          3|          0|   N|        TOO SMALL!!!|it´s nice BUT it´...|
|R3LCIANTN1H9EC|B001H4BQ1A|          4|          1|   N|          Four Stars|It does the job b...|
|R3U2M23N1

In [60]:
import pyspark.sql.functions as F
product_id_count= review_id_table_df.groupBy('product_id').count()
product_id_count.sort(col("count").desc()).show(truncate=False)
# product_id_count.show()

+----------+-----+
|product_id|count|
+----------+-----+
|B004EZG9X4|1948 |
|B00A8SXGKA|1534 |
|B00HSWJUI6|1384 |
|B00KKXCJQU|1351 |
|B004XAVT8K|1074 |
|B000SKXRNO|887  |
|B007B5POVC|839  |
|B0082C62BO|623  |
|B000SKXROI|545  |
|B001TR0UZI|514  |
|B004AB4YWA|503  |
|B004AB70US|502  |
|B006HMPV4A|497  |
|B002VS8H3G|479  |
|B00AIJ57OW|478  |
|B000P8JBPC|467  |
|B002B3FWXY|457  |
|B005KDBHWM|450  |
|B000YB7T04|448  |
|B000F1RGBU|428  |
+----------+-----+
only showing top 20 rows



In [49]:
product_id_count.count()

64584

In [52]:
# -- Customer table for first data set
counts_df = review_id_table_df.groupBy('vine').count()
counts_df.show()

+----+------+
|vine| count|
+----+------+
|   Y|   904|
|   N|347709|
+----+------+



In [53]:
# Start Spark session
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("StopWords").getOrCreate()

In [54]:
 # import stopwords library
from pyspark.ml.feature import StopWordsRemover

In [55]:
from pyspark.sql.functions import concat_ws

review_id_table_df.withColumn("review_headline", concat_ws("", "review_headline")).show()


+--------------+----------+-----------+-----------+----+--------------------+--------------------+
|     review_id|product_id|star_rating|total_votes|vine|     review_headline|         review_body|
+--------------+----------+-----------+-----------+----+--------------------+--------------------+
| R9CO86UUJCAW5|B00VGTN02Y|          3|          0|   N|my review of this...|my review of this...|
|R3PR8X6QGVJ8B1|B005KIWL0E|          5|          0|   N|          Five Stars|       Perfect size.|
|R39BO2819ABUPF|B007UNSHJ6|          4|          0|   N|            so good.|    So far, so good.|
|R3ADL7V6EGGEEP|B00WHFNXB4|          4|          0|   N|          Four Stars|seems nicely made...|
|R1OXYPBPLVRMI5|B000M5NBYU|          5|          0|   N|           Dance Bag|Great bag for all...|
|R1WYM8Z5ATQ98O|B00ACBJ89G|          3|          0|   N|        TOO SMALL!!!|it´s nice BUT it´...|
|R3LCIANTN1H9EC|B001H4BQ1A|          4|          1|   N|          Four Stars|It does the job b...|
|R3U2M23N1

In [56]:
from pyspark.sql.functions import split, col
df2 =review_id_table_df.select(split(col("review_headline")," ").alias("review_headline_tokens")) \
    .drop("review_headline")
df2.printSchema()
df2.show()

root
 |-- review_headline_tokens: array (nullable = true)
 |    |-- element: string (containsNull = true)

+----------------------+
|review_headline_tokens|
+----------------------+
|  [my, review, of, ...|
|         [Five, Stars]|
|           [so, good.]|
|         [Four, Stars]|
|          [Dance, Bag]|
|       [TOO, SMALL!!!]|
|         [Four, Stars]|
|         [Five, Stars]|
|  [We, were, exited...|
|               [cute!]|
|           [One, Star]|
|  [A, great, bag, f...|
|             [Perfect]|
|  [it's, durable, a...|
|  [Strong, well-mad...|
|         [Five, Stars]|
|        [Three, Stars]|
|        [Three, Stars]|
|    [Laptop, backpack]|
|           [Fabulous!]|
+----------------------+
only showing top 20 rows

